In [2]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

# 1. Create a function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wiki_file, kaggle_file, ratings_file):
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    file_dir = "C:/Users/rob/OneDrive/Desktop/Data Bootcamp/Movies-ETL"
    kaggle_metadata = pd.read_csv(f'{file_dir}/Resources/movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv(f'{file_dir}/kaggle_movies/ratings.csv')

    # 3. Open the read the Wikipedia data JSON file.
    with open(f'{file_dir}/Resources/wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
    # 4. Read in the raw wiki movie data as a Pandas DataFrame.
    wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    # 5. Return the three DataFrames
    return wiki_movies_df, kaggle_metadata, ratings

# 6 Create the path to your file directory and variables for the three files. 
file_dir = "C:/Users/rob/OneDrive/Desktop/Data Bootcamp/Movies-ETL"
# Wikipedia data
wiki_file = f'{file_dir}/Resources/wikipedia_movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/Resources/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/kaggle_movies/ratings.csv'

# 7. Set the three variables in Step 6 equal to the function created in Step 1.
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file, kaggle_file, ratings_file)

In [3]:
# 8. Set the DataFrames from the return statement equal to the file names in Step 6. 
wiki_movies_df = wiki_file
kaggle_metadata = kaggle_file
ratings = ratings_file

In [9]:
# 9. Check the wiki_movies_df DataFrame.
wiki_movies_df.sample(10)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
4979,https://en.wikipedia.org/wiki/Welcome_Home_Ros...,2008.0,https://www.imdb.com/title/tt0494652/,Welcome Home Roscoe Jenkins,Malcolm D. Lee,"[Scott Stuber, Mary Parent, Charles Castaldi]",NaN,NaN,NaN,"[Martin Lawrence, Margaret Avery, Joy Bryant, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6448,https://en.wikipedia.org/wiki/Sin_City:_A_Dame...,2014.0,https://www.imdb.com/title/tt0458481/,Sin City: A Dame to Kill For,"[Robert Rodriguez, Frank Miller]","[Robert Rodriguez, Aaron Kaufman, Stephen L'He...",NaN,NaN,"[Sin City, by Frank Miller]","[Mickey Rourke, Jessica Alba, Josh Brolin, Jos...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4246,https://en.wikipedia.org/wiki/The_Holiday,2006.0,https://www.imdb.com/title/tt00457939/,The Holiday,Nancy Meyers,"[Nancy Meyers, Bruce A. Block]",NaN,NaN,NaN,"[Kate Winslet, Cameron Diaz, Jude Law, Jack Bl...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
650,https://en.wikipedia.org/wiki/South_Central_(f...,1992.0,https://www.imdb.com/title/tt0105450/,South Central,Stephen Milburn Anderson,"[William B. Steakley, Oliver Stone, Janet Yang]",NaN,NaN,NaN,"[Glenn Plummer, Carl Lumbly]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6836,https://en.wikipedia.org/wiki/Fist_Fight,2017.0,https://www.imdb.com/title/tt3401882/,Fist Fight,Richie Keen,"[Shawn Levy, Max Greenfield, John Rickard, Dan...","[Van Robichaux, Evan Susser]","[Van Robichaux, Evan Susser, Max Greenfield]",NaN,"[Ice Cube, Charlie Day, Tracy Morgan, Jillian ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5139,https://en.wikipedia.org/wiki/New_Moon_(2009_f...,2009.0,https://www.imdb.com/title/tt1259571/,The Twilight Saga: New Moon,Chris Weitz,"[Wyck Godfrey, Karen Rosenfelt]",Melissa Rosenberg,NaN,"[New Moon, by, Stephenie Meyer]","[Kristen Stewart, Robert Pattinson, Taylor Lau...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1669,https://en.wikipedia.org/wiki/Hype!,1996.0,https://www.imdb.com/title/tt0116589/,Hype!,Doug Pray,Steven Helvey,NaN,NaN,NaN,Numerous musicians,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2325,https://en.wikipedia.org/wiki/The_Impostors,1998.0,https://www.imdb.com/title/tt0120823/,The Impostors,Stanley Tucci,"[Elizabeth W. Alexander,, Stanley Tucci]",NaN,NaN,NaN,"[Oliver Platt, Stanley Tucci, Teagle F. Bouger...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4835,https://en.wikipedia.org/wiki/The_Family_That_...,2008.0,https://www.imdb.com/title/tt1142798/,Tyler Perry's The Family That Preys,Tyler Perry,Tyler Perry,NaN,NaN,NaN,"[Alfre Woodard, Kathy Bates, Sanaa Lathan, Roc...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6042,https://en.wikipedia.org/wiki/The_Fifth_Estate...,2013.0,https://www.imdb.com/title/tt1837703/,The Fifth Estate,Bill Condon,"[Steve Golin, Brad Dorros, Michael Sugar]",Josh Singer,NaN,"[Inside WikiLeaks, by, Daniel Domscheit-Berg, ...","[Benedict Cumberbatch, Daniel Brühl, Anthony M...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# 10. Check the kaggle_metadata DataFrame.
kaggle_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
# 11. Check the ratings DataFrame.
ratings

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990
